In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import os, sys
sys.path.append('../pubmed_rag')
#from utils import get_chunks
from bioc import (
    mean_pooling
)


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/anglup/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


get the data for the vx db first. 

Note: The primary key and vector fields use their default names (“id” and “vector”).

to insert into vx database need each row as a dictionary

In [3]:
# read in
filename = '/Users/anglup/GitHub/pubmed-rag/biocjson/all_embeddings.csv'
df = pd.read_csv(filename, )
# rename id and vector cols
df = df.rename(
    columns={
        'Unnamed: 0':'id',
        'embedding':'vector'
    }, 
)
# ensure vector is a list
df['vector'] = df['vector'].apply(literal_eval)

# sanity check
#data.head()

# to list of dicts
data = df.to_dict(orient='records')

In [44]:
df.head()

,id,section,pmid,pmcid,date,authors,journal,text,vector
0,0,abstract,28924171,PMC5603535,2017-09-18 00:00:00+00:00,Luo Y and Zhao X and Zhou J and Yang J and Zha...,Nat Commun,"The emergence of large-scale genomic, chemical...","[-0.027584053575992584, 0.054618917405605316, ..."
1,1,discuss,28924171,PMC5603535,2017-09-18 00:00:00+00:00,Luo Y and Zhao X and Zhou J and Yang J and Zha...,Nat Commun,Discussion . Recent advances in large-scale ex...,"[0.00014715130964759737, 0.054599590599536896,..."
2,2,discuss,28924171,PMC5603535,2017-09-18 00:00:00+00:00,Luo Y and Zhao X and Zhou J and Yang J and Zha...,Nat Commun,"On the other hand, based on the systematic int...","[0.00014715130964759737, 0.054599590599536896,..."
3,3,discuss,28924171,PMC5603535,2017-09-18 00:00:00+00:00,Luo Y and Zhao X and Zhou J and Yang J and Zha...,Nat Commun,Our findings are particularly helpful for unde...,"[0.00014715130964759737, 0.054599590599536896,..."
4,4,intro,28924171,PMC5603535,2017-09-18 00:00:00+00:00,Luo Y and Zhao X and Zhou J and Yang J and Zha...,Nat Commun,Introduction . Computational prediction of dru...,"[-0.039880603551864624, -0.011334901675581932,..."


In [4]:
from pymilvus import MilvusClient

In [37]:
client = MilvusClient('demo.db')

if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
)


In [39]:
result = client.insert(collection_name="demo_collection", data=data)
print(result)


{'insert_count': 47, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46]}


yay first vx db created

In [5]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

chosen_model = 'sentence-transformers/all-mpnet-base-v2'

#logger.info(f"Loading model {chosen_model} from HuggingFace")
tokenizer = AutoTokenizer.from_pretrained(chosen_model)
model = AutoModel.from_pretrained(chosen_model)

/Users/anglup/Library/Caches/pypoetry/virtualenvs/pubmed-rag-iB5Z1S4t-py3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# Tokenize sentences
encoded_input = tokenizer(
    "Can you please tell me what nodes and edges I should include in a biological knowledge graph for drug repurposing?", 
    padding=True, 
    truncation=False, 
    return_tensors='pt'
)

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[ 3.2914e-02, -3.1554e-02,  7.1610e-03, -9.6834e-03, -2.0804e-02,
         -2.9990e-02, -2.1607e-02,  2.8089e-02,  8.4443e-02, -2.1861e-03,
          2.7061e-02,  7.4580e-02, -8.7139e-03,  9.1949e-02,  8.1665e-03,
          4.3600e-02,  6.0810e-03,  5.2895e-02,  9.5613e-03, -4.6859e-02,
         -2.3129e-02,  1.7358e-02,  1.5377e-03, -1.5145e-03, -7.0703e-03,
          1.4116e-03,  8.3259e-03,  2.2533e-02,  1.5436e-02, -4.8137e-02,
         -1.2683e-02, -1.7956e-02, -1.6711e-02,  8.5764e-03,  1.2389e-06,
         -4.5091e-02, -5.2964e-02,  3.6673e-02, -4.8655e-02, -3.8745e-02,
          5.4656e-02, -4.1287e-02,  5.5994e-02, -3.5536e-02,  7.0920e-03,
         -4.3131e-02,  1.9771e-02,  2.5608e-02,  2.1471e-02,  1.9946e-02,
         -3.8807e-03, -9.9989e-03,  7.1122e-02, -2.4125e-02,  2.6173e-02,
         -1.0321e-02,  5.3093e-02,  1.8854e-02,  5.6223e-02, -1.3137e-02,
         -6.1156e-03,  3.4451e-02, -3.0967e-02, -2.8824e-02,  1.1615e-01,
         -2.5856e

In [10]:
client = MilvusClient('../demo.db')

result = client.search(
    collection_name="main",
    data=sentence_embeddings.detach().numpy(),
    #filter="section == 'abstract'",
    limit=10,
    output_fields=["text", "pmid", 'section'],
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [45]:
result[0]

[{'id': 593,
  'distance': 0.7601258754730225,
  'entity': {'section': 'title',
   'pmid': 35840060,
   'text': 'KG-Predict: A knowledge graph computational framework for drug repurposing.'}},
 {'id': 426,
  'distance': 0.724739134311676,
  'entity': {'section': 'title',
   'pmid': 34297570,
   'text': 'Knowledge Graph-Based Approaches to Drug Repurposing for COVID-19.'}},
 {'id': 534,
  'distance': 0.6885125041007996,
  'entity': {'section': 'title',
   'pmid': 34181736,
   'text': 'CROssBAR: comprehensive resource of biomedical relations with knowledge graph representations.'}},
 {'id': 649,
  'distance': 0.6845976114273071,
  'entity': {'section': 'methods',
   'pmid': 36175836,
   'text': 'To the extent that it incorporates multiple graph structural variations, RTX-KG2 can also be used as a test-bed for evaluating the performance of structurally generalizable graph analysis methods such as a subset of us have done for the case of a structurally generalizable node-node similarity me

In [38]:
import json

In [41]:
for x in (result[0]):
    print(json.dumps(x, indent=4))

{
    "id": 593,
    "distance": 0.7601258754730225,
    "entity": {
        "section": "title",
        "pmid": 35840060,
        "text": "KG-Predict: A knowledge graph computational framework for drug repurposing."
    }
}
{
    "id": 426,
    "distance": 0.724739134311676,
    "entity": {
        "section": "title",
        "pmid": 34297570,
        "text": "Knowledge Graph-Based Approaches to Drug Repurposing for COVID-19."
    }
}
{
    "id": 534,
    "distance": 0.6885125041007996,
    "entity": {
        "section": "title",
        "pmid": 34181736,
        "text": "CROssBAR: comprehensive resource of biomedical relations with knowledge graph representations."
    }
}
{
    "id": 649,
    "distance": 0.6845976114273071,
    "entity": {
        "section": "methods",
        "pmid": 36175836,
        "text": "To the extent that it incorporates multiple graph structural variations, RTX-KG2 can also be used as a test-bed for evaluating the performance of structurally generalizable g